# Annual Daylight Results Mesh Analysis

## Overview
This script takes summarised annual daylight results meshes (eg: Daylight Autonomy) and compares the impact of different features (eg: size of skylight 1 vs size of skylight 2).

In [93]:
import os
import numpy as np
import sklearn
import pandas as pd


## Prepare data objects
First we create a dictionary with empty numpy arrays ready to receive data. The dictionary is ordered as follows:

- Daylight Metric 1 (eg: daylight autonomy)
    - Room 1
        - empty ndarray
    - Room 2
        - empty ndarray
        
- Daylight Metric 2 (eg: udli)
    - Room 1
        - empty ndarray
    - Room 2
        - empty ndarray

In [94]:
path = "C:/Users/adao/Desktop/Warwick_Daylighting/Processed/"


floors = {}
skylight_data = {}
dla_data = {}
udli_low_data = {}
udli_data = {}
udli_high_data = {}

for file in os.listdir(path):
    parameters = file.split("_")
    floor = parameters[0]
    
    text = open(path + file)
    content = text.readlines()
    mesh_size = len(content[0].split(","))
    
    floors[floor] = []
    skylight_data[floor] = []
    dla_data[floor] = []
    udli_low_data[floor] = []
    udli_data[floor] = []
    udli_high_data[floor] = []
    
sDAlist = []


## Import the data
For each daylight metric and floor we extract the data from the results files and append them to the correct dictionary key pairs

In [95]:
def get_row(content,i):
    row = content[i].split(",")
    try:
        row[-1] = row[-1].split('\n')[0]
    except:
        pass
    return row

In [96]:
i = 0
for file in os.listdir(path):
    text = open(path + file)
    content = text.readlines()
    
    parameters = file.split("_")
    
    floor = parameters[0]
    west_skylight = float(parameters[2])# + "." + parameters[5])
    south_skylight = float(parameters[4])# + "." + parameters[8])
    north_skylight = float(parameters[6].split('.')[0])# + "." + parameters[11].split(".")[0])

    skylight_datum = [west_skylight, south_skylight, north_skylight]

    skylight_data[floor].append(skylight_datum)
    
    dla = get_row(content, 0)

    udli_low = get_row(content, 1)

    udli = get_row(content, 2)

    udli_high = get_row(content, 3)

    

    dla_data[floor].append(dla)

    udli_low_data[floor].append(udli_low)

    udli_data[floor].append(udli)

    udli_high_data[floor].append(udli_high)

    if floor == '1st':
        i+=1
        
    # Get sDA data into sDAList to then put into pandas
    sDA = content[4]
    try:
        sDa = sDa.split('\n')[0]
    except:
        pass
    entry = {'west_skylight':west_skylight, 'south_skylight':south_skylight, 'north_skylight':north_skylight,
            floor:float(sDA)}
    sDAlist.append(entry)

for floor in list(skylight_data.keys()):
    skylight_data[floor] = np.asarray(skylight_data[floor])
    print(skylight_data[floor].shape)
    dla_data[floor] = np.asarray(dla_data[floor]).astype('float64')
    print(dla_data[floor].shape)
    udli_low_data[floor] = np.asarray(udli_low_data[floor]).astype('float64')
    print(udli_low_data[floor].shape)
    udli_data[floor] = np.asarray(udli_data[floor]).astype('float64')
    print(udli_data[floor].shape)
    udli_high_data[floor] = np.asarray(udli_high_data[floor]).astype('float64')
    print(udli_high_data[floor].shape)


(64, 3)
(64, 4633)
(64, 4633)
(64, 4633)
(64, 4633)
(64, 3)
(64, 4294)
(64, 4294)
(64, 4294)
(64, 4294)
(64, 3)
(64, 3050)
(64, 3050)
(64, 3050)
(64, 3050)
(64, 3)
(64, 2760)
(64, 2760)
(64, 2760)
(64, 2760)
(64, 3)
(64, 2954)
(64, 2954)
(64, 2954)
(64, 2954)
(64, 3)
(64, 3176)
(64, 3176)
(64, 3176)
(64, 3176)
(64, 3)
(64, 1641)
(64, 1641)
(64, 1641)
(64, 1641)


In [97]:
df = pd.DataFrame(sDAlist)
# df.groupby(['north_skylight','south_skylight','west_skylight']).mean()
# print(df)
df.to_csv("C:/Users/adao/Desktop/Warwick_Daylighting/SVM Results/sDA.csv")

In [98]:
floor = 'Ground'
print(floor + " " + str(dla_data[floor][0,:]))
print(floor + " " + str(dla_data[floor][60,:]))
# for floor in daylight_data['udli_low'].keys():
#     print(floor + " " + str(metric_meshes['dla'][floor]))

Ground [ 90.  90.  91. ...,   6.  20.  58.]
Ground [ 90.  90.  91. ...,   9.  22.  54.]


## Initialise SVM
We use sci-kit learn One Classe Support Vector Machine (SVM) algorithm to...

In [99]:
from sklearn.svm import OneClassSVM
def get_coefficients(X,y):
    clf = OneClassSVM(kernel='linear')
    clf.fit(X,y)
    return clf.coef_

In [100]:
from sklearn.preprocessing import MinMaxScaler

def get_coeffs(resultsDict):
    for floor in list(resultsDict.keys()):
        print(floor + " " + str(resultsDict[floor].shape))
        mesh = np.array([]).reshape(0,3)
                
        for Y in np.transpose(resultsDict[floor]):
            X = skylight_data[floor]

            coefs = get_coefficients(X,Y)
            #print(coefs.shape)
            #scaler = MinMaxScaler()
            #scaler.fit(coefs)
            #coefs = scaler.transform(coefs)
            stdev = np.std(Y)
            impact = stdev * np.asarray([[1,1,1]]) #coefs * stdev
            
            mesh = np.append(mesh, impact, axis=0)
        
        mesh = np.transpose(mesh)
        
        print("~~~~ Output mesh shape " + str(mesh.shape))
        print("")

        resultsDict[floor] = mesh
    
    return mesh.shape

In [101]:
print(get_coeffs(dla_data))
print("~~~~ New Metric ~~~~~")
print(get_coeffs(udli_low_data))
print("~~~~ New Metric ~~~~~")
print(get_coeffs(udli_data))
print("~~~~ New Metric ~~~~~")
print(get_coeffs(udli_high_data))

1st (64, 4633)
~~~~ Output mesh shape (3, 4633)

2nd (64, 4294)
~~~~ Output mesh shape (3, 4294)

3rd (64, 3050)
~~~~ Output mesh shape (3, 3050)

4th (64, 2760)
~~~~ Output mesh shape (3, 2760)

5th (64, 2954)
~~~~ Output mesh shape (3, 2954)

Ground (64, 3176)
~~~~ Output mesh shape (3, 3176)

Mezzanine (64, 1641)
~~~~ Output mesh shape (3, 1641)

(3, 1641)
~~~~ New Metric ~~~~~
1st (64, 4633)
~~~~ Output mesh shape (3, 4633)

2nd (64, 4294)
~~~~ Output mesh shape (3, 4294)

3rd (64, 3050)
~~~~ Output mesh shape (3, 3050)

4th (64, 2760)
~~~~ Output mesh shape (3, 2760)

5th (64, 2954)
~~~~ Output mesh shape (3, 2954)

Ground (64, 3176)
~~~~ Output mesh shape (3, 3176)

Mezzanine (64, 1641)
~~~~ Output mesh shape (3, 1641)

(3, 1641)
~~~~ New Metric ~~~~~
1st (64, 4633)
~~~~ Output mesh shape (3, 4633)

2nd (64, 4294)
~~~~ Output mesh shape (3, 4294)

3rd (64, 3050)
~~~~ Output mesh shape (3, 3050)

4th (64, 2760)
~~~~ Output mesh shape (3, 2760)

5th (64, 2954)
~~~~ Output mesh shap

In [102]:
def saveToTxt(data,name):
    for floor in skylight_data.keys():
            datum = data[floor]
            print(floor + " " + str(datum.shape))
            file = "C:/Users/adao/Desktop/Warwick_Daylighting/SVM Results/" + name + "_" + floor +".txt" 
            np.savetxt(file,datum, delimiter=",")

In [103]:
saveToTxt(dla_data, "sda")
saveToTxt(udli_data, "udli")
saveToTxt(udli_low_data, "udli_low")
saveToTxt(udli_high_data, "udli_high")

1st (3, 4633)
2nd (3, 4294)
3rd (3, 3050)
4th (3, 2760)
5th (3, 2954)
Ground (3, 3176)
Mezzanine (3, 1641)
1st (3, 4633)
2nd (3, 4294)
3rd (3, 3050)
4th (3, 2760)
5th (3, 2954)
Ground (3, 3176)
Mezzanine (3, 1641)
1st (3, 4633)
2nd (3, 4294)
3rd (3, 3050)
4th (3, 2760)
5th (3, 2954)
Ground (3, 3176)
Mezzanine (3, 1641)
1st (3, 4633)
2nd (3, 4294)
3rd (3, 3050)
4th (3, 2760)
5th (3, 2954)
Ground (3, 3176)
Mezzanine (3, 1641)


In [104]:
import numpy as np

def pareto_frontier_multi(myArray):
    # Sort on first dimension
    myArray = myArray[myArray[:,0].argsort()]
    # Add first row to pareto_frontier
    pareto_frontier = myArray[0:1,:]
    # Test next row against the last row in pareto_frontier
    for row in myArray[1:,:]:
        if sum([row[x] >= pareto_frontier[-1][x]
                for x in range(len(row))]) == len(row):
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier = np.concatenate((pareto_frontier, [row]))
    return pareto_frontier

def test():
    myArray = np.array([[1,1,1],[2,2,2],[4,4,4],[3,3,3],[3,2,3]])
    print(pareto_frontier_multi(myArray))

test()

[[1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]]
